<a href="https://colab.research.google.com/github/mario-rot/Introduction-to-Human-Language-Technology/blob/main/Session7_LaurenTucker_MarioRosas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 7 (Word Sequences) - ILTH

**Students:** Lauren Tucker & Mario Rosas !!!

## Paraphrases Template

In [ ]:
%%shell
git clone https://github.com/mario-rot/Introduction-to-Human-Language-Technology.git
cd 'Introduction-to-Human-Language-Technology'
mv 'Complementary Material' /content/

Cloning into 'Introduction-to-Human-Language-Technology'...
remote: Enumerating objects: 849, done.
remote: Counting objects: 100% (849/849), done.
remote: Compressing objects: 100% (777/777), done.
remote: Total 849 (delta 107), reused 788 (delta 67), pack-reused 0
Receiving objects: 100% (849/849), 2.25 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [ ]:
import pandas as pd

In [ ]:
dt = pd.read_csv('Complementary Material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [ ]:
dt.head()

,0,1
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi..."


In [ ]:
dt['gs'] = pd.read_csv('Complementary Material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

In [ ]:
dt.shape

(459, 3)

In [ ]:
dt.head()

,0,1,gs
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.50
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.00
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.25
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.50
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.00


# Excercise

In [ ]:
from nltk.metrics import jaccard_distance
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from scipy.stats import pearsonr
import numpy as np

import string
import collections
import nltk
from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
from nltk.corpus import gutenberg
from nltk.text import Text
nltk.download('gutenberg')
nltk.download('stopwords')

def remove_signs(wrd):
  wrd = list(wrd)
  wrd = [word for word in wrd if not any(caracter in signs for caracter in word)]
  wrd = ''.join(wrd)
  return wrd

def clean(corpus, stopwords, minwords_len, signs):
   corpus = corpus.split(' ')
   corpus = [word.lower() for word in corpus]
   corpus = [word if not any(caracter in signs for caracter in word) else remove_signs(word) for word in corpus]
   corpus = [word for word in corpus if word not in stopwords and word.isalpha()]
   corpus = [word for word in corpus if len(word) > minwords_len]
   return corpus

from nltk.corpus.reader.wordnet import VERB, NOUN, ADJ, ADV
pos_map = {'N': NOUN,
           'V':VERB,
           'J':ADJ,
           'R':ADV}

# corpus = list(gutenberg.words('blake-poems.txt'))
stopwords=set(nltk.corpus.stopwords.words('english'))
signs = string.punctuation
minwords_len = 2

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
cleaned = clean(dt.loc[0][0], stopwords, minwords_len, signs)
cleaned

['leaders', 'given', 'new', 'chance', 'let', 'hope', 'seize']

In [ ]:
accepted_pos = ['JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBN', 'VBP', 'VBZ']

pos_map = {'n': ['NN', 'NNS', 'NNP', 'NNPS'],
           'v': ['VB', 'VBD', 'VBN', 'VBP', 'VBZ'],
           'j': ['JJ', 'JJR', 'JJS'],
           'r': ['RB', 'RBR', 'RBS']}
correcting = {'n':'n', 'v':'v', 'j':'a', 'r':'r'}
pos_map2 = {'N': NOUN,
           'V':VERB,
           'J':ADJ,
           'R':ADV}


def filter_pos(pair):
    if pair[1][0].lower() in list(pos_map.keys()):
        return pair[0], correcting[pair[1][0].lower()]
    return None, None

# def filter_pos(pair):
#     if pair[1] in accepted_pos:
#         return pair
#     return None, None

In [ ]:
def applying_lesk(sentence):
    cleaned = clean(sentence, stopwords, minwords_len, signs)
    pairs = nltk.pos_tag(cleaned)
    synsets = []
    for pair in pairs:
      context = cleaned
      word, pos = filter_pos(pair)
      if pos:
        synset = nltk.wsd.lesk(context, word, pos)
      else:
          synset = False
      if synset:
        synsets.append(synset)
    return synsets


In [ ]:
wnl = nltk.stem.WordNetLemmatizer()


def lemmatize(p):
    if p[1][0] in {'N', 'V', 'J', 'R'}:
        return wnl.lemmatize(p[0].lower(), pos=pos_map2[p[1][0]])
    return p[0]


def lemmatize_sentence(sentence):
    cleaned = clean(sentence, stopwords, minwords_len, signs)
    tagged = nltk.pos_tag(cleaned)
    return [[lemmatize(pair),pair[1]] for pair in tagged]


def applying_lesk_lemmas(sentence):
    context = [i[0] for i in sentence]
    synsets = []
    for pair in sentence:
        context = cleaned
        word, pos = filter_pos(pair)
        if pos:
            synset = nltk.wsd.lesk(context, word, pos)
        else:
            synset = False
        if synset:
            synsets.append(synset)
    return synsets

In [ ]:
def compute_metric(metric, elements): # Fenction to calculate Jaccard Distance
    if metric == 'jaccard':
        res = jaccard_distance(set(applying_lesk(elements[0])),
                               set(applying_lesk(elements[1])))
    if metric == 'jaccard_lemmas':
      res = jaccard_distance(set(applying_lesk_lemmas(lemmatize_sentence(elements[0]))),
                             set(applying_lesk_lemmas(lemmatize_sentence(elements[1]))))
    return res

In [ ]:
dt['jaccard'],dt['jaccard_lemmas'] = "",""

In [ ]:
for i in range(len(dt)): # Iterating to get Jaccard distances over each column of the dataframe
  dt['jaccard'][i] = np.float64(compute_metric('jaccard',[dt.iloc[i][0],dt.iloc[i][1]]))*10
  dt['jaccard_lemmas'][i] = np.float64(compute_metric('jaccard_lemmas',[dt.iloc[i][0],dt.iloc[i][1]]))*10

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
dt

,0,1,gs,jaccard,jaccard_lemmas
0,The leaders have now been given a new chance a...,The leaders benefit aujourd' hui of a new luck...,4.500,5.555556,5.555556
1,Amendment No 7 proposes certain changes in the...,Amendment No 7 is proposing certain changes in...,5.000,0.0,0.0
2,Let me remind you that our allies include ferv...,I would like to remind you that among our alli...,4.250,7.5,7.5
3,The vote will take place today at 5.30 p.m.,The vote will take place at 5.30pm,4.500,2.5,2.5
4,"The fishermen are inactive, tired and disappoi...","The fishermen are inactive, tired and disappoi...",5.000,0.0,0.0
...,...,...,...,...,...
454,It is our job to continue to support Latvia wi...,It is of our duty of continue to support the c...,5.000,6.363636,6.363636
455,The vote will take place today at 5.30 p.m.,Vote will take place at 17 h 30.,4.750,2.5,2.5
456,Neither was there a qualified majority within ...,There was no qualified majority in this Parlia...,5.000,5.0,5.0
457,Let me remind you that our allies include ferv...,"I hold you recall that our allies, there are e...",4.000,7.5,7.5


In [ ]:
pearsonr(dt['gs'], 1-dt['jaccard']) # Calculating the pearson correlation between GS results and 1-Jaccard calculated data with lesk algorithm applied

(0.45497704134035827, 7.827296967470252e-25)

In [ ]:
pearsonr(dt['gs'], 1-dt['jaccard_lemmas']) # Calculating the pearson correlation between GS results and 1-Jaccard calculated data with lesk algorithm + lemmatizer applied

(0.45658512629245984, 5.110934549149753e-25)

In [ ]:
count = 0
for index, (j,jl) in enumerate(zip(dt['jaccard'],dt['jaccard_lemmas'])):
    if j != jl:
        count += 1
        print(index,'-->', j,'--',jl)

8 --> 7.777777777777778 -- 6.25
14 --> 10.0 -- 8.75
62 --> 10.0 -- 8.75
80 --> 10.0 -- 8.75
100 --> 7.0 -- 5.555555555555555
118 --> 8.0 -- 6.666666666666666
175 --> 10.0 -- 5.0
184 --> 5.555555555555555 -- 3.75
187 --> 5.7142857142857135 -- 3.333333333333333
188 --> 6.25 -- 4.285714285714286
209 --> 6.666666666666666 -- 0.0
235 --> 7.777777777777778 -- 6.25
236 --> 10.0 -- 8.88888888888889
242 --> 7.0 -- 5.555555555555555
255 --> 8.0 -- 6.666666666666666
256 --> 7.0 -- 5.555555555555555
265 --> 6.666666666666666 -- 5.0
285 --> 2.8571428571428568 -- 0.0
287 --> 10.0 -- 5.0
323 --> 7.777777777777778 -- 6.25
343 --> 5.0 -- 2.8571428571428568
347 --> 6.25 -- 4.285714285714286
404 --> 10.0 -- 8.88888888888889
410 --> 4.285714285714286 -- 1.6666666666666665


# Conclusion
For reference, the Pearson correlation for Lab 2 was:

*PearsonRResult(statistic=0.45049771693186835, pvalue=2.5356459143049236e-24)*

and the Pearson correlation for Lab 3 was:

*PearsonRResult(statistic=0.48102317341708245, pvalue=5.904510415498371e-28)*


In this lab, we first removed stopwords and punctuation from our original sentences and filtered with pos_tag to receive pairs of significant words and their corresponding parts of speech for all nouns, adjectives, adverbs, and verbs. We then applied the lesk algorithm to this cleaned set of words and received a Pearson correlation value of:

*PearsonRResult(statistic=0.454977041340358, pvalue=7.827296967470474e-25)*

We then took an additional step and lemmatized the cleaned set of words and performed lesk on this lemmatized set, and we received a Pearson correlation value of:

*PearsonRResult(statistic=0.4565851262924598, pvalue=5.110934549149753e-25)*


Lesk, both when performed on the lemmatized set and the unlemmatized set, performed better than the document analysis from Lab 2. In both cases, this is likely because the context that the Lesk algorithm uses is supporting the evaluation of word similarities and creates a more accurate picture of the meaning of each word. Lesk occasionally performed better on the lemmatized set than on the unlemmatized set, with 24 of the instances having lower Jaccard coefficients for the lemmatized sets. Contrary to our expectations, the lemmatization does not seem to have a negative impact on the context. In other words, lemmatizing the sentence does not strip any meaning from the context. In any case, however, the improvement in performance when using Lesk is very small, despite our predictions that the improvement would be much greater, since we hypothesized that having context would improve performance.


Lesk, both when performed on the lemmatized set and the unlemmatized set, performed worse than just lemmatizing the words as done in Lab 3, as shown above. One possible reason for this may be that, when taking more words into account when determining similarity (i.e. by adding the context), there is more room for error. If the wrong synset is determined for some word in the context, then it will impact the interpreted meaning of the whole sentence. With just lemmatizing, on the other hand, as done in Lab 3, there is less chance of error, since only the structure of each word is being changed and examined, without the subjective interpretations of meaning used with context analysis.







